In [ ]:
from PyQt5 import QtWidgets
from PyQt5 import QtCore
from PyQt5 import QtGui
import win32gui
import win32ui
import win32con 
import cv2
import numpy as np
import time
import colorsys

In [ ]:
LABEL_LIST = ['현재색(F5)', '보색', '저장A(F2)', '저장B(F3)', 'A+B 혼합', '3x3 평균', 'HSV']

# 배경지식--

## 파이썬으로 ColorPicker를 구현하기 위해서 다른 라이브러리를 사용하지 않고 윈도우 API를 사용한다 (속도때문)
## pypiwin32 라이브러리 사용

## 윈도우 프로그래밍
### GetDesktopWindow():
####                                     Windows OS에서 최상위 윈도우를 구하는 함수
####                                     최상위 윈도우는 모든 윈도우의 부모 윈도우를 구하는 함수


### Handle:
####                                      Windows OS에서 생선되는 자원에 부여하는 고유값
####                                      핸들의 포괄적인 목적은 해당 자원을 핸들링하기 위함


### Device Context:
####                                      Windows OS에서 화면출력에 관한 모든 정보를 담고 있는 구조체
####                                      Windows에서 글자를 쓰던 그림을 그리던 DC를 이용해야 함


In [ ]:
def captureWindow(x=0, y=0, width=0, height=0):
    hwin = win32gui.GetDesktopWindow()
    hwindc = win32gui.GetWindowDC(hwin)
    srcdc = win32ui.CreateDCFromHandle(hwindc) # 현재 보여지는 화면 --> 여기에 그리면 실제로 그려짐
    memdc = srcdc.CreateCompatibleDC() # 복사본 생성 후 작업
    bmp = win32ui.CreateBitmap()
    bmp.CreateCompatibleBitmap(srcdc, width, height)  # 압축되지 않은 원본 이미지
    memdc.SelectObject(bmp)
    memdc.BitBlt((0,0), (width, height), srcdc, (x, y), win32con.SRCCOPY)
    sigIntArray = bmp.GetBitmapBits(True)
    img = np.fromstring(sigIntArray, dtype='uint8')
    img.shape = (height, width, 4) # 4채널 RGBA
    
    srcdc.DeleteDC()
    memdc.DeleteDC()
    win32gui.ReleaseDC(hwin, hwindc)
    win32gui.DeleteObject(bmp.GetHandle())
    
    return cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

In [ ]:
class MyPicker(QtWidgets.QMainWindow):
    def __init__(self):
        super().__init__()
        
        self.zoom = 1
        self.pause = False
        self.saveBGR = [(-1, -1, -1), (-1, -1, -1)]
        self.title = '파이피커'
        self.setWindowTitle(self.title)
        self.setFixedSize(300, 500)
        self.main_widget = QtWidgets.QWidget()
        self.createWidgets()
        self.setCentralWidget(self.main_widget)
        
        self.worker = Worker()
        self.worker.sig_update.connect(self.updateWindow)
        self.worker.start()
        self.show()
       
    def keyPressEvent(self, e):
        if e.key() == QtCore.Qt.Key_F2:
            self.saveBGR[0] = self.currentBGR
        elif e.key() == QtCore.Qt.Key_F3:
            self.saveBGR[1] = self.currentBGR
        elif e.key() == QtCore.Qt.Key_F5:
            self.pause = not self.pause
        elif e.key() == QtCore.Qt.Key_F7:
            self.zoom += 1
        elif e.key() == QtCore.Qt.Key_F8:
            self.zoom -= 1
            self.zoom < 1: self.zoom = 1
            
            
    @QtCore.pyqtSlot(tuple)
    def updateWindow(self, pt):
        x, y = pt
        #print(x, y)
        width = self.viewer.width()
        height = self.viewer.height()
        
        center_x = int(width / 2)
        center_y = int(height / 2)
        
        x -= center_x
        y -= center_y
        
        if x < 0: x = 0
        if y < 0: y = 0
            
        display = []
        capture = captureWindow(x=x, y=y, width=width, height=height)
        
        if self.zoom > 1:
            zoom_image = self.resize(caputre, width=self.zoom * capture.shape[1])
            if zoom_image is not None:
                center = int(zoom_image.shape[0] / 2), int(zoom_image.shape[1] / 2)
                # 이미지 크롭 [y1:y2, x1:x2]
                capture = zoom_image[center[0] - center_y:center[0] + center_ym center[1] - center_x:center[1] + center_x]
        # 화면 중앙 십자가 그리기
        # 빨간색 적용
        bgr = capture[center_y, center_x]
        self.currentBGR = (bgr[0], bgr[1], bgr[2])
        
        # hsv = 색상 (Hue), 채도(Saturation), 명도(Value)
        #       hsv는 1.0을 기준으로 하기 때문에 1로 만들기 위해 /255
        #                             R            G           B    
        hsv = colorsys.rgb_to_hsv(bgr[2]/255, bgr[1]/255, bgr[0]/255)
        # hsv --> photoshop 형태로 표기
        hsv_angle = (int(hsv[0] * 360), int(hsv[1] * 100), int(hsv[2] * 100))
        # 보색
        bgr_inverse = self.get_inverse_color(bgr)
        
        out_code, text_color, bg_color = self.get_code(bgr)
        out_inverse, text_color_inverse, bg_color_inverse = self.get_code(bgr_inverse)
        
        out_hsv = f'{hsv_angle[0]}°, {hsv_angle[1]}% {hsv_angle[2]}%'
        
        aR = aG = aB = 0
        for i in range(center_y - 1, cneter_y + 2):
            for j in range(center_x - 1, center_x + 2):
                p = capture[i, j]
                aR += p[2]
                aG += p[1]
                aB += p[0]
        aR = abs(int(aR / 9))
        aG = abs(int(aG / 9))
        aB = abs(int(aB / 9))
    
        out_3x3, text_color_3x3, bg_color_3x3 = self.get_code((aB, aG, aR))
        
        out_save1 = out_save2 = out_mix = ''
        text_color_save1 = text_color_save2 = text_color_mix = '#000000'
        bg_color_save1 = bg_color_save2 = bg_color_mix = '#FFFFFF'
        
        if self.saveBGR[0][0] >= 0:
            out_save1, text_color_save1, bg_color_save1 = self.get_code(self.saveBGR[0])
        if self.saveBGR[1][0] >= 0:
            out_save2, text_color_save2, bg_color_save2 = self.get_code(self.saveBGR[1])
            
        # 색상 혼합 공식: A+B = A+((B-A/Z)*1)
        if self.saveBGR[0][0] >= 0 and self.saveBGR[1][0] >= 0:
            nR = self.saveBGR[0][2] + (int((self.BGR[1][2] - self.saveBGR[0][2]) / 2) * 1)
            nG = self.saveBGR[0][1] + (int((self.BGR[1][1] - self.saveBGR[0][1]) / 2) * 1)
            nB = self.saveBGR[0][0] + (int((self.BGR[1][0] - self.saveBGR[0][0]) / 2) * 1)
            out_mix, text_color_mix, bg_color_mix = self.get_code((nB, nG, nR))
        
        
        display.append((out_code, text_color, bg_color))
        display.append((out_inverse, text_color_inverse, bg_color_inverse))
        display.append((out_save1, text_color_save1, bg_color_save1))
        display.append((out_save2, text_color_save2, bg_color_save2))
        display.append((out_mix, text_color_mix, bg_color_mix))
        display.append((out_color_3x3, text_color_3x3, bg_color_3x3))
        display.append((out_hsv, '#FFFFFF', '#000000'))
    
        for i, c in enumerate(self.colors):
            text, text_color, bg_color = display[i]
            style = 'background-color:' + bgcolor + '; color:' + text_color
            c.setText(text)
    
        step = int(width / 10)
        # 전체 화면에 20% 크기로 세팅
        cv2.line(capture, (center_x - step, center_y), (center_x + step, center_y), (255, 255, 255), 1)
        cv2.line(capture, (center_x, center_y - step), (center_x, center_y + step), (255, 255, 255), 1)
        qImage = QtGui.QImage(capture.data, width, height, 3 * width, QtGui.QImage.Format_RGB888)
        pixmap = QtGui.QPixmap.fromImage(qImage)
        self.viewer.setPixmap(pixmap)
       
    def resize(self, image, width=None, height=None, inter=cv2.INTER_AREA):
        if width is None and height is None:
            return None
        dim = None
        (h, w) = image.shape[:2]
        if width is None:
            r = height / float(h)
            dim = (int(w * r), height)
        else:
            r = width / float(w)
            dim = (width, int(h * r))
            
        resized = cv2.resize(image, dim, interpolation=inter)
        return resized
            
        
        
    def get_code(self, bgr):
        inverse = self.get_inverse_color(bgr)
        current_text = self.comboBox.currentText()
        out_text = ''
        if current_text == 'HTML':
            out_text = '#%02x%02x%02x' %(bgr[2], bgr[1], bgr[0])
        elif current_text == 'RGB':
            out_text = f'{bgr[2]}, {bgr[1]}, {bgr[0]}'
            
        bg_color = '#%02x%02x%02x' %(bgr[2], bgr[1], bgr[0])
        text_color = '#%02x%02x%02x' %(inverse[2], inverse[1], inverse[0])
        return (out_text, text_color, bg_color)
        
    def get_inverse_color(self, bgr):
        return (abs(255 - bgr[0]), (abs(255 - bgr[1]), (abs(255 - bgr[2])
        
    def createWidgets(self):
        self.labelStyle = 'padding:1px; font-size:13px; font-family:맑은 고딕;'
        self.editStyle = 'padding:1px; font-size:13px; font-family:맑은 고딕; border:1px solid #000000'
        
        vbox = QtWidgets.QVBoxLayout()
        self.viewer = QtWidgets.QLabel()
        self.viewer.setBaseSize(300, 300)
        self.viewer.setText('이미지창')
        #                        가로로 가운데정렬   | 두가지 옵션 모두 적용    세로로 가운데 정렬
        self.viewer.setAlignment(QtCore.Qt.AlignCenter|QtCore.Qt.AlignVCenter)
        vbox.addWidget(self.viewer)
        
        grid = QtWidgets.QGridLayout()
        grid.setSpacing(3)
        
        self.labels = []
        self.colors = []
                                                        
        for i, l in enumerate(LABEL_LIST):
            label = QtWidgets.QLabel(l)
            label.setStyleSheet(self.labelStyle)
            label.setAlignment(QtCore.Qt.AlignRight | QtCore.Qt.AlignVCenter)
            
            color = QtWidgets.QLineEdit()
            color.setStyleSheet(self.editStyle)
            color.setReadOnly(True)
            
            grid.addWidget(label, i + 1, 0)
            grid.addWidget(color, i + 1, 1)
                                                        
            self.labels.append(label)
            self.colors.append(color)
        
        label = QtWidgets.QLabel('코드형식')
        label.setStyleSheet(self.labelStyle)
        label.setAlignment(QtCore.Qt.AlignRight | QtCore.Qt.AlignVCenter)
        
        self.comboBox = QtWidgets.QComboBox()
        self.comboBox.addItem('HTML')
        self.comboBox.addItem('RGB')
        self.comboBox.setStyleSheet('padding:3px;')
        grid.addWidget(label, len(LABEL_LIST) + 1, 0)
        grid.addWidget(self.comboBox, len(LABEL_LIST) + 1, 1)
        
        vbox.addLayout(grid)
        self.main_widget.setLayout(vbox)

In [ ]:
class Worker(QtCore.QThread):
    sig_update = QtCore.pyqtSignal(tuple)
    
    def __init__(self):
        super(Worker, self).__init__()
        
    # Override
    def run(self):
        while True:
            # 현재 마우스의 좌표
            pt = win32gui.GetCursorPos()
            self.sig_update().emit(pt)
            time.sleep(0.1)

In [ ]:
app = QtWidgets.QApplication([])
picker = MyPicker()
app.exec_()
# c = captureWindow(0, 0, 200, 200)
# cv2.imshow('S', c)
# cv2.waitKey(0)
# cv2.destroyAllWindows()